<a href="https://colab.research.google.com/github/yg-li/QA-KG-RL/blob/master/QA_KG_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --verbose --no-cache-dir torch-scatter torch-sparse torch-cluster torch-spline-conv
!pip install torch-geometric
# !pip install dgl

In [0]:
import torch
from torch_geometric.data import Data
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.get_default_dtype())